In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
inputs = keras.Input(shape=(198,3), name="digits")
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = layers.Flatten()(x)
x = layers.Dropout(.5)(x)
x = layers.Dense(951, activation="relu", name="dense_1")(x)  # 951
x = layers.Dropout(.5)(x)
x = layers.Dense(951, activation="relu", name="dense_2")(x) # 951 v6:500
x = layers.Dropout(.5)(x)
x = layers.Dense(951, activation="relu", name="dense_3")(x) # 951 v6:500
outputs = layers.Dense(951, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 198, 3)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 196, 64)           640       
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dropout (Dropout)            (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 951)               11930295  
_________________________________________________________________
dropout_1 (Dropout)          (None, 951)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 951)               905352

In [3]:
x_train=np.load("3-mer.npy")
x_test = np.load("3-mer_test.npy")
names=open("npy_2/name.txt",'r')
y=[]
y_e=[]
p=0
for i in names.readlines():
  j=[float(p)]*100
  q=[float(p)]*20
  y+=j
  y_e+=q
  p+=1
names.close()
y_train = np.array(y).reshape((95100,1)).astype("float32")
y_test = np.array(y_e).reshape((19020,1)).astype("float32")
x_train = x_train.astype("float32")/4
x_test = x_test.astype("float32")/4

In [4]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.05, momentum=0.1, nesterov=False, name="SGD"),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,name='sparse_categorical_crossentropy'),
    metrics=['accuracy'],
)

In [5]:
print("Fit model on training data")

history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=25,
    shuffle = True,
    validation_data=(x_test, y_test),
)

Fit model on training data
Epoch 1/25
743/743 [==============================] - 15s 12ms/step - loss: 6.7499 - accuracy: 0.0018 - val_loss: 6.3494 - val_accuracy: 0.0033
Epoch 2/25
743/743 [==============================] - 9s 12ms/step - loss: 5.8470 - accuracy: 0.0083 - val_loss: 5.3153 - val_accuracy: 0.0160
Epoch 3/25
743/743 [==============================] - 8s 11ms/step - loss: 5.2518 - accuracy: 0.0192 - val_loss: 4.9940 - val_accuracy: 0.0387
Epoch 4/25
743/743 [==============================] - 9s 12ms/step - loss: 4.8367 - accuracy: 0.0553 - val_loss: 4.2590 - val_accuracy: 0.1721
Epoch 5/25
743/743 [==============================] - 9s 12ms/step - loss: 3.7873 - accuracy: 0.2252 - val_loss: 2.6635 - val_accuracy: 0.4890
Epoch 6/25
743/743 [==============================] - 9s 11ms/step - loss: 2.4646 - accuracy: 0.4815 - val_loss: 1.6932 - val_accuracy: 0.6880
Epoch 7/25
743/743 [==============================] - 8s 11ms/step - loss: 1.7483 - accuracy: 0.6304 - val_loss: 1

In [6]:
model.save("my_model_v7.h5")

In [7]:
model = keras.models.load_model("my_model_v7.h5")

def QMer(seq):
    q=3
    data = []
    for i in range(len(seq)-(q-1)):
        data.append(list(seq[i:i+q]))
    return data

In [8]:
rna = input("RNA:").strip()
p=[]
for i in rna:
    k=0
    if(i=="A"):
        k=1
    elif(i=='U' or i=="T"):
        k=2
    elif(i=='C'):
        k=3
    elif(i=='G'):
        k=4
    else:
        k=0
    p.append(k)
p=p+[0]*(200-len(p))
p=QMer(p)
b=np.array(p)
b=b.reshape((1,198,3))/4
names=open("npy_2/name.txt",'r')
t=model.predict(b)
t1=np.argmax(t)
o = names.readlines()
pred = o[t1][:-5]
print(pred)
names.close()

RNA:GTTTCAATCCCCTCGCGGGGATTTTGGTTTTTTGAAG
RF01334
